In [ ]:
import plotly.express as px
import plotly.io as pio
pio.templates.default = "plotly_white"   # semua chart default putih


#library yang digunakan untuk memanipulasi data
import pandas as pd #digunakan untuk membaca dan mengelola dataset
import numpy as np #digunakan untuk mengoprasikan numerik dasar

#library yang digunakan untuk memvisualisasi data
import seaborn as sns #digunakan untuk membuat heatmap
import matplotlib.pyplot as plt #digunakan untuk membuat canvas plot matplotlib
import plotly.express as px #digunakan untuk membuat diagram batang interaktif

#library yang digunakan untuk antarmuka pengguna
import streamlit as st  #digunakan untuk membuat aplikasi web interaktif

#library yang digunakan untuk visualisasi kata
from wordcloud import WordCloud #digunakan untuk membuat WordCloud dari teks keterampilan

#library yang digunakan untuk representasi vektor dan kemiripan
from sentence_transformers import SentenceTransformer #digunakan untuk menghasilkan embedding SBERT
from sklearn.metrics.pairwise import cosine_similarity #digunakan untuk menghitung kemiripan antar vektor
from sklearn.feature_extraction.text import TfidfVectorizer #digunakan untuk mengubah teks menjadi vektor TF-IDF

#library yang digunakan untuk teks dan sistem
import re #digunakan untuk preprocessing teks  menggunakan regex
import io #digunakan untuk kebutuhan ekspor file
import warnings #digunakan untuk menonaktifkan peringatan selama eksekusi dijalankan
warnings.filterwarnings("ignore")


In [ ]:
# STREAMLIT UI SETUP
st.set_page_config(page_title="Sistem Rekomendasi Kandidat", layout="wide")
st.title("Sistem Rekomendasi Berdasarkan Keterampilan")

In [ ]:
# CUSTOM BACKGROUND DAN WARNA TEKS
st.markdown("""
    <style>
    html, body, [class*="css"]  {
        background-color: white !important;
        color: black !important;
    }
    </style>
""", unsafe_allow_html=True)



In [ ]:
# ==========================================
# UPLOAD DATASET
# ==========================================
st.sidebar.header("Upload Dataset")
candidate_file = st.sidebar.file_uploader("Upload Dataset Kandidat (CSV)", type="csv")
job_file = st.sidebar.file_uploader("Upload Dataset Lowongan (CSV)", type="csv")

if candidate_file and job_file:
    candidate_df = pd.read_csv(candidate_file)
    job_df = pd.read_csv(job_file)

    st.subheader("Dataset Kandidat")
    st.dataframe(candidate_df.head())
    st.subheader("Dataset Lowongan Pekerjaan")
    st.dataframe(job_df.head())

    # ==========================================
    # CLEANING & PREPROCESSING
    # ==========================================
    st.subheader("Pembersihan dan Preprocessing Kandidat")
    awal = candidate_df.shape[0]
    candidate_df.drop_duplicates(inplace=True)
    candidate_df.dropna(subset=['clean_skills'], inplace=True)
    akhir = candidate_df.shape[0]
    st.write(f"Jumlah data sebelum dibersihkan: {awal}")
    st.write(f"Jumlah data setelah dibersihkan: {akhir}")

    st.subheader("Pembersihan dan Preprocessing Lowongan")
    awal_job = job_df.shape[0]
    job_df.drop_duplicates(inplace=True)
    job_df.dropna(subset=['IT Skills', 'Soft Skills'], how='all', inplace=True)
    akhir_job = job_df.shape[0]
    st.write(f"Jumlah data sebelum dibersihkan: {awal_job}")
    st.write(f"Jumlah data setelah dibersihkan: {akhir_job}")

    def preprocess(text):
        text = str(text).lower()
        text = re.sub(r"[^a-zA-Z\s]", "", text)
        text = re.sub(r"\s+", " ", text)
        return text.strip()

    candidate_df['raw_skills'] = candidate_df['clean_skills']  # Simpan sebelum preprocessing
    candidate_df['clean_skills'] = candidate_df['clean_skills'].apply(preprocess)
    candidate_df['position'] = candidate_df['position'].fillna('').apply(preprocess)
    candidate_df['Experience'] = candidate_df['Experience'].fillna('').apply(preprocess)
    candidate_df['description'] = candidate_df['description'].fillna('').apply(preprocess)

    job_df['raw_required_skills'] = job_df['IT Skills'].fillna('') + ' ' + job_df['Soft Skills'].fillna('')
    job_df['required_skills'] = job_df['raw_required_skills'].apply(preprocess)

    # ==========================================
    # WORDCLOUD KANDIDAT (BEFORE & AFTER)
    # ==========================================
    st.subheader("WordCloud Kandidat")
    col1, col2 = st.columns(2)

    with col1:
        st.markdown("#### Sebelum Preprocessing")
        raw_skills = " ".join(candidate_df['raw_skills'].dropna().astype(str))
        wc_raw = WordCloud(width=400, height=300, background_color='white').generate(raw_skills)
        st.image(wc_raw.to_array(), use_container_width=True)

    with col2:
        st.markdown("#### Setelah Preprocessing")
        clean_skills = " ".join(candidate_df['clean_skills'].dropna().astype(str))
        wc_clean = WordCloud(width=400, height=300, background_color='white').generate(clean_skills)
        st.image(wc_clean.to_array(), use_container_width=True)

    # ==========================================
    # WORDCLOUD LOWONGAN (BEFORE & AFTER)
    # ==========================================
    st.subheader("WordCloud Lowongan")
    col3, col4 = st.columns(2)

    with col3:
        st.markdown("#### Sebelum Preprocessing")
        raw_lowongan = job_df['raw_required_skills'].dropna().astype(str)
        wc_job_raw = WordCloud(width=400, height=300, background_color='white').generate(" ".join(raw_lowongan))
        st.image(wc_job_raw.to_array(), use_container_width=True)

    with col4:
        st.markdown("#### Setelah Preprocessing")
        clean_lowongan = job_df['required_skills'].dropna().astype(str)
        wc_job_clean = WordCloud(width=400, height=300, background_color='white').generate(" ".join(clean_lowongan))
        st.image(wc_job_clean.to_array(), use_container_width=True)

    # ==========================================
    # REKOMENDASI
    # ==========================================
    st.subheader("Pilih Lowongan untuk Rekomendasi Kandidat")
    job_titles = job_df['Job Title'].dropna().unique().tolist()
    selected_job = st.selectbox("Pilih Job Title:", job_titles)

    metode = st.radio("Pilih Metode Pencocokan:", ["TF-IDF", "Sentence-BERT", "Hybrid"])

    if selected_job:
        selected_job_desc = job_df[job_df['Job Title'] == selected_job]['required_skills'].values[0]

        if metode in ["TF-IDF", "Hybrid"]:
            tfidf_vectorizer = TfidfVectorizer()
            tfidf_matrix = tfidf_vectorizer.fit_transform(candidate_df['clean_skills'])
            job_tfidf_vec = tfidf_vectorizer.transform([selected_job_desc])
            tfidf_sim = cosine_similarity(tfidf_matrix, job_tfidf_vec).flatten()
            candidate_df['tfidf_sim'] = tfidf_sim

        if metode in ["Sentence-BERT", "Hybrid"]:
            model = SentenceTransformer('all-MiniLM-L6-v2')
            combined = candidate_df['clean_skills'] + " " + candidate_df['position'] + " " + candidate_df['Experience'] + " " + candidate_df['description']
            embeddings = model.encode(combined.tolist(), show_progress_bar=True)
            job_emb = model.encode([selected_job_desc])[0].reshape(1, -1)
            sbert_sim = cosine_similarity(embeddings, job_emb).flatten()
            candidate_df['sbert_sim'] = sbert_sim

        if metode == "TF-IDF":
            candidate_df['final_score'] = candidate_df['tfidf_sim']
        elif metode == "Sentence-BERT":
            candidate_df['final_score'] = candidate_df['sbert_sim']
        else:
            candidate_df['final_score'] = 0.5 * candidate_df['tfidf_sim'] + 0.5 * candidate_df['sbert_sim']

        # ==========================================
        # TOP-N KANDIDAT
        # ==========================================
        st.subheader("Top Kandidat Sesuai Lowongan")
        top_n = st.slider("Tampilkan Top-N Kandidat", 1, 20, 10)
        top_candidates = candidate_df.sort_values(by='final_score', ascending=False).head(top_n)
        top_candidates = top_candidates[top_candidates['Name'].notna()]
        top_candidates['Name'] = top_candidates['Name'].astype(str)

        st.dataframe(top_candidates[['Name', 'position', 'clean_skills', 'final_score']])

        # ==========================================
        # VISUALISASI
        # ==========================================
        st.subheader("Visualisasi Skor Kandidat")
        fig = px.bar(top_candidates, x='Name', y='final_score', color='position', title="Skor Kecocokan Kandidat")
        st.plotly_chart(fig)

        if metode != "TF-IDF":
            st.subheader("Heatmap Kemiripan antar Kandidat")
            sim_matrix = cosine_similarity(embeddings)
            fig2, ax = plt.subplots(figsize=(10, 6))
            sns.heatmap(sim_matrix[:top_n, :top_n], cmap='plasma', xticklabels=False, yticklabels=False)
            st.pyplot(fig2)

        # ==========================================
        # TOMBOL DOWNLOAD & RATING MANUAL
        # ==========================================
        st.download_button("Download Hasil Rekomendasi", top_candidates.to_csv(index=False), "hasil_rekomendasi.csv", "text/csv")
        st.subheader("Evaluasi Manual Rekomendasi")
        for i, row in top_candidates.iterrows():
            st.markdown(f"**{row['Name']} - {row['position']}**")
            rating = st.slider(f"Seberapa cocok kandidat ini?", 1, 5, 3, key=f"rating_{i}")

else:
    st.warning("Silakan unggah kedua dataset (kandidat dan lowongan kerja).")
